<h1><center>MPST: A Corpus of Movie Plot Synopses with Tags</center></h1>

In [1]:
!pip install scikit-multilearn

import re
import os
import tqdm
import nltk
import pickle
import sqlite3
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import xgboost as xgb
import tensorflow as tf
from sklearn import metrics
from tensorflow import keras
from nltk.corpus import words
from datetime import datetime
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from itertools import combinations
from keras.models import load_model
from keras.models import Sequential
from tensorflow.keras import layers
from nltk.stem import SnowballStemmer
from sklearn.pipeline import Pipeline
from nltk.tokenize import sent_tokenize
from keras.preprocessing import sequence
from scipy.sparse import coo_matrix, hstack
from tensorflow.keras.utils import plot_model
from keras.layers.embeddings import Embedding
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import RandomizedSearchCV
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score,precision_score,recall_score,hamming_loss
from keras.layers import Conv1D, Conv2D, Dense, Dropout, Flatten, LSTM, GlobalMaxPooling1D, MaxPooling2D, Activation, BatchNormalization

%matplotlib inline
nltk.download('punkt')
nltk.download('wordnet')
warnings.filterwarnings("ignore")
stemmer = SnowballStemmer('english')

%autosave 120

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

Autosaving every 120 seconds


In [2]:
data_with_all_tags = pd.read_csv("data_with_all_tags.csv")
data_with_all_tags.head()

,title,plot_synopsis,tags,split,synopsis_source,cnt_dup,tag_count,synopsis_count,synopsis_sent_count,CleanedSynopsis,...,sentimental,storytelling,stupid,suicidal,suspenseful,thought-provoking,tragedy,violence,western,whimsical
0,$,"Set in Hamburg, West Germany, several criminal...",murder,test,imdb,1,1,648,26,set hamburg west germani sever crimin take adv...,...,0,0,0,0,0,0,0,0,0,0
1,$windle,A 6th grader named Griffin Bing decides to gat...,flashback,train,wikipedia,1,1,353,14,grader name griffin bing decid gather entir gr...,...,0,0,0,0,0,0,0,0,0,0
2,'71,"Gary Hook, a new recruit to the British Army, ...","suspenseful, neo noir, murder, violence",train,wikipedia,1,4,699,39,gari hook new recruit british armi take leav m...,...,0,0,0,0,1,0,0,1,0,0
3,'A' gai wak,Sergeant Dragon Ma (Jackie Chan) is part of th...,"cult, violence",train,wikipedia,1,2,665,41,sergeant dragon jacki chan part hong kong mari...,...,0,0,0,0,0,0,0,1,0,0
4,'Breaker' Morant,"In Pretoria, South Africa, in 1902, Major Char...","murder, anti war, violence, flashback, tragedy...",train,wikipedia,1,6,1694,140,pretoria south africa major charl bolton rod m...,...,0,0,0,0,0,0,1,1,0,0


In [3]:
conn = sqlite3.connect('data.db')
data_with_all_tags.to_sql('data', conn, if_exists='replace', index=False)
train = pd.read_sql("Select * From data where split = 'train' OR split='val'",conn)
test =  pd.read_sql("Select * From data where split = 'test'",conn)
conn.close()

In [4]:
X_train = train["CleanedSynopsis"]
y_train= train["tags"]

X_test = test["CleanedSynopsis"]
y_test= test["tags"]

In [5]:
def tokenize(x):
    x=x.split(',')
    tags=[i.strip() for i in x] #Some tags contains whitespaces before them, so we need to strip them
    return tags

cnt_vectorizer = CountVectorizer(tokenizer = tokenize, max_features=3, binary='true').fit(y_train)
y_train_multilabel = cnt_vectorizer.transform(y_train)
y_test_multilabel = cnt_vectorizer.transform(y_test)

<h1> 1. TfidfVectorizer with 1 grams: </h1>

In [6]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,1))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 657) Y : (10989, 3)
Dimensions of test data X: (2768, 657) Y: (2768, 3)


<h2> 1.1 OneVsRestClassifier + MultinomialNB:</h2>

In [7]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [8]:
prediction1 = clf.predict(X_test_multilabel)

precision1 = precision_score(y_test_multilabel, prediction1, average='micro')

recall1 = recall_score(y_test_multilabel, prediction1, average='micro')

f1_score1 = 2*((precision1 * recall1)/(precision1 + recall1))

print("precision1: {:.4f}, recall1: {:.4f}, F1-measure: {:.4f}".format(precision1, recall1, f1_score1))

precision1: 0.4674, recall1: 0.6864, F1-measure: 0.5561


In [9]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction1[k])[0],"\n")

Movie:  Anastasia
Actual genre:  romantic, cute, entertaining
Predicted tag:  [] 

Movie:  Dog City
Actual genre:  psychedelic
Predicted tag:  [] 

Movie:  Aftermath
Actual genre:  violence
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  What a Nightmare, Charlie Brown!
Actual genre:  psychedelic
Predicted tag:  [] 

Movie:  Witchcraft 7: Judgement Hour
Actual genre:  paranormal
Predicted tag:  ['flashback' 'murder' 'violence'] 



<h2> 1.2 OneVsRestClassifier + SGDClassifier with LOG Loss:</h2>

In [10]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [11]:
prediction2 = clf.predict(X_test_multilabel)

precision2 = precision_score(y_test_multilabel, prediction2, average='micro')

recall2 = recall_score(y_test_multilabel, prediction2, average='micro')

f1_score2 = 2*((precision2 * recall2)/(precision2 + recall2))

print("precision2: {:.4f}, recall2: {:.4f}, F1-measure: {:.4f}".format(precision2, recall2, f1_score2))

precision2: 0.5057, recall2: 0.6680, F1-measure: 0.5757


In [12]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction2[k])[0],"\n")

Movie:  Capitalism: A Love Story
Actual genre:  sentimental
Predicted tag:  ['flashback'] 

Movie:  Saved!
Actual genre:  romantic, humor, satire
Predicted tag:  [] 

Movie:  How to Train Your Dragon 2
Actual genre:  revenge, cute
Predicted tag:  ['violence'] 

Movie:  Orfeu Negro
Actual genre:  atmospheric
Predicted tag:  ['murder'] 

Movie:  Accepted
Actual genre:  romantic, action
Predicted tag:  [] 



<h2> 1.3 OneVsRestClassifier + SGDClassifier with Hinge Loss:</h2>

In [13]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [14]:
prediction3 = clf.predict(X_test_multilabel)

precision3 = precision_score(y_test_multilabel, prediction3, average='micro')

recall3 = recall_score(y_test_multilabel, prediction3, average='micro')

f1_score3 = 2*((precision3 * recall3)/(precision3 + recall3))

print("precision3: {:.4f}, recall3: {:.4f}, F1-measure: {:.4f}".format(precision3, recall3, f1_score3))

precision3: 0.5110, recall3: 0.6608, F1-measure: 0.5763


In [15]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction3[k])[0],"\n")

Movie:  Assassin's Creed IV: Black Flag
Actual genre:  action
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  The Kite Runner
Actual genre:  romantic, violence, murder, storytelling, flashback
Predicted tag:  ['flashback' 'violence'] 

Movie:  Guest House Paradiso
Actual genre:  psychedelic, comedy
Predicted tag:  [] 

Movie:  Gingerdead Man 2: Passion of the Crust
Actual genre:  comedy, murder, violence
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Zazie dans le métro
Actual genre:  absurd, psychedelic
Predicted tag:  [] 



<h2> 1.4  OneVsRestClassifier + LogisticRegression:</h2>

In [16]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [17]:
prediction4 = clf.predict(X_test_multilabel)

precision4 = precision_score(y_test_multilabel, prediction4, average='micro')

recall4 = recall_score(y_test_multilabel, prediction4, average='micro')

f1_score4 = 2*((precision4 * recall4)/(precision4 + recall4))

print("precision4: {:.4f}, recall4: {:.4f}, F1-measure: {:.4f}".format(precision4, recall4, f1_score4))

precision4: 0.5076, recall4: 0.6836, F1-measure: 0.5826


In [18]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction4[k])[0],"\n")

Movie:  The Sting
Actual genre:  boring, depressing, murder, cult, plot twist, clever, inspiring, revenge, entertaining
Predicted tag:  ['flashback' 'murder'] 

Movie:  Olivia
Actual genre:  neo noir, cruelty, murder, violence, revenge, sadist
Predicted tag:  [] 

Movie:  El capo
Actual genre:  satire
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Bereavement
Actual genre:  cult
Predicted tag:  ['murder' 'violence'] 

Movie:  Arpointeu
Actual genre:  neo noir, murder, paranormal, violence, flashback, revenge
Predicted tag:  ['flashback' 'murder' 'violence'] 



<h1>2. TfidfVectorizer with (1 - 2 Grams):<?h1>

In [19]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,2))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 3)
Dimensions of test data X: (2768, 666) Y: (2768, 3)


<H2> 2.1 OneVsRestClassifier + MultinomialNB :</H2>

In [20]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [21]:
prediction5 = clf.predict(X_test_multilabel)

precision5 = precision_score(y_test_multilabel, prediction5, average='micro')

recall5 = recall_score(y_test_multilabel, prediction5, average='micro')

f1_score5 = 2*((precision5 * recall5)/(precision5 + recall5))

print("precision5: {:.4f}, recall5: {:.4f}, F1-measure: {:.4f}".format(precision5, recall5, f1_score5))

precision5: 0.4680, recall5: 0.6860, F1-measure: 0.5564


In [22]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction5[k])[0],"\n")

Movie:  Long-Distance Princess
Actual genre:  romantic
Predicted tag:  ['flashback'] 

Movie:  Time Lapse
Actual genre:  plot twist, mystery, murder
Predicted tag:  ['murder'] 

Movie:  What's Eating Gilbert Grape
Actual genre:  tragedy, whimsical, psychedelic, boring, depressing
Predicted tag:  ['flashback'] 

Movie:  The Rebound
Actual genre:  comedy, romantic
Predicted tag:  ['flashback'] 

Movie:  Une liaison pornographique
Actual genre:  pornographic, realism, flashback
Predicted tag:  ['flashback'] 



<h2> 2.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</h2>

In [23]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [24]:
prediction6 = clf.predict(X_test_multilabel)

precision6 = precision_score(y_test_multilabel, prediction6, average='micro')

recall6 = recall_score(y_test_multilabel, prediction6, average='micro')

f1_score6 = 2*((precision6 * recall6)/(precision6 + recall6))

print("precision6: {:.4f}, recall6: {:.4f}, F1-measure: {:.4f}".format(precision6, recall6, f1_score6))

precision6: 0.5160, recall6: 0.6776, F1-measure: 0.5858


In [25]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction6[k])[0],"\n")

Movie:  Cape Fear
Actual genre:  suspenseful, gothic, murder, neo noir, mystery, violence, cult, horror, flashback, good versus evil, revenge
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Wild Bill
Actual genre:  cult, revenge, storytelling, flashback
Predicted tag:  ['violence'] 

Movie:  Beyond the Law
Actual genre:  romantic, neo noir, murder, violence, flashback
Predicted tag:  ['murder'] 

Movie:  Blancanieves
Actual genre:  cruelty, murder
Predicted tag:  [] 

Movie:  Gabriel Over the White House
Actual genre:  depressing
Predicted tag:  ['violence'] 



<h2> 2.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [26]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [27]:
prediction7 = clf.predict(X_test_multilabel)

precision7 = precision_score(y_test_multilabel, prediction7, average='micro')

recall7 = recall_score(y_test_multilabel, prediction7, average='micro')

f1_score7 = 2*((precision7 * recall7)/(precision7 + recall7))

print("precision7: {:.4f}, recall7: {:.4f}, F1-measure: {:.4f}".format(precision7, recall7, f1_score7))

precision7: 0.4907, recall7: 0.7023, F1-measure: 0.5777


In [28]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction7[k])[0],"\n")

Movie:  Les deux orphelines vampires
Actual genre:  insanity
Predicted tag:  ['murder'] 

Movie:  The Blob
Actual genre:  violence, cult, suspenseful, comedy, murder
Predicted tag:  ['murder'] 

Movie:  La casa muda
Actual genre:  plot twist, revenge
Predicted tag:  [] 

Movie:  Fist Fight
Actual genre:  prank
Predicted tag:  [] 

Movie:  Linewatch
Actual genre:  good versus evil, violence
Predicted tag:  ['murder' 'violence'] 



<h2> 2.4 OneVsRestClassifier + LogisticRegression:</h2>

In [29]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [30]:
prediction8 = clf.predict(X_test_multilabel)

precision8 = precision_score(y_test_multilabel, prediction8, average='micro')

recall8 = recall_score(y_test_multilabel, prediction8, average='micro')

f1_score8 = 2*((precision8 * recall8)/(precision8 + recall8))

print("precision8: {:.4f}, recall8: {:.4f}, F1-measure: {:.4f}".format(precision8, recall8, f1_score8))

precision8: 0.5081, recall8: 0.6848, F1-measure: 0.5834


In [31]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction8[k])[0],"\n")

Movie:  Hare and Loathing in Las Vegas
Actual genre:  psychedelic
Predicted tag:  [] 

Movie:  Misconduct
Actual genre:  neo noir
Predicted tag:  ['murder' 'violence'] 

Movie:  Ten Little Indians
Actual genre:  murder
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Hostel
Actual genre:  boring, gothic, murder, stupid, violence, cult, horror, action, revenge, sadist
Predicted tag:  ['violence'] 

Movie:  Scooby-Doo and the Witch's Ghost
Actual genre:  paranormal, psychedelic, horror, mystery
Predicted tag:  [] 



<h1>3. TfidfVectorizer with (1 - 3 Grams):<?h1>

In [32]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,3))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 3)
Dimensions of test data X: (2768, 666) Y: (2768, 3)


<H2> 3.1 OneVsRestClassifier + MultinomialNB :</H2>

In [33]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [34]:
prediction9 = clf.predict(X_test_multilabel)

precision9 = precision_score(y_test_multilabel, prediction9, average='micro')

recall9 = recall_score(y_test_multilabel, prediction9, average='micro')

f1_score9 = 2*((precision9 * recall9)/(precision9 + recall9))

print("precision9: {:.4f}, recall9: {:.4f}, F1-measure: {:.4f}".format(precision9, recall9, f1_score9))

precision9: 0.4680, recall9: 0.6860, F1-measure: 0.5564


In [35]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction9[k])[0],"\n")

Movie:  Tezaab
Actual genre:  revenge, romantic, flashback
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  My Summer of Love
Actual genre:  dramatic, romantic, queer
Predicted tag:  ['flashback'] 

Movie:  The Ten Commandments: The Musical
Actual genre:  historical fiction
Predicted tag:  ['violence'] 

Movie:  The Gambler and the Lady
Actual genre:  revenge, murder
Predicted tag:  ['murder' 'violence'] 

Movie:  Roxanne
Actual genre:  romantic, psychedelic, entertaining
Predicted tag:  ['flashback'] 



<H2> 3.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</H2>

In [36]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [37]:
prediction10 = clf.predict(X_test_multilabel)

precision10 = precision_score(y_test_multilabel, prediction10, average='micro')

recall10 = recall_score(y_test_multilabel, prediction10, average='micro')

f1_score10 = 2*((precision10 * recall10)/(precision10 + recall10))

print("precision10: {:.4f}, recall10: {:.4f}, F1-measure: {:.4f}".format(precision10, recall10, f1_score10))

precision10: 0.5174, recall10: 0.6688, F1-measure: 0.5835


In [38]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction10[k])[0],"\n")

Movie:  American Son
Actual genre:  violence, romantic, flashback
Predicted tag:  ['flashback'] 

Movie:  Conan the Destroyer
Actual genre:  murder, cult, fantasy, alternate history, violence
Predicted tag:  ['murder' 'violence'] 

Movie:  Love & Other Drugs
Actual genre:  romantic
Predicted tag:  ['flashback'] 

Movie:  The Gazebo
Actual genre:  suspenseful, murder
Predicted tag:  ['murder'] 

Movie:  Kuch Kuch Hota Hai
Actual genre:  flashback
Predicted tag:  ['flashback'] 



<h2> 3.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [39]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [40]:
prediction11 = clf.predict(X_test_multilabel)

precision11 = precision_score(y_test_multilabel, prediction11, average='micro')

recall11 = recall_score(y_test_multilabel, prediction11, average='micro')

f1_score11 = 2*((precision11 * recall11)/(precision11 + recall11))

print("precision11: {:.4f}, recall11: {:.4f}, F1-measure: {:.4f}".format(precision11, recall11, f1_score11))

precision11: 0.4960, recall11: 0.6864, F1-measure: 0.5758


In [41]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction11[k])[0],"\n")

Movie:  Austin Powers: International Man of Mystery
Actual genre:  comedy, boring, cult, good versus evil, psychedelic, humor, satire, revenge, entertaining
Predicted tag:  [] 

Movie:  American Dreamer
Actual genre:  intrigue
Predicted tag:  ['flashback'] 

Movie:  Shootout at Wadala
Actual genre:  violence
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Haunted - 3D
Actual genre:  paranormal
Predicted tag:  ['flashback'] 

Movie:  All-Star Superman
Actual genre:  tragedy, whimsical, romantic, action
Predicted tag:  ['flashback'] 



<h2> 3.4 OneVsRestClassifier + LogisticRegression:</h2>

In [42]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [43]:
prediction12 = clf.predict(X_test_multilabel)

precision12 = precision_score(y_test_multilabel, prediction12, average='micro')

recall12 = recall_score(y_test_multilabel, prediction12, average='micro')

f1_score12 = 2*((precision12 * recall12)/(precision12 + recall12))

print("precision12: {:.4f}, recall12: {:.4f}, F1-measure: {:.4f}".format(precision12, recall12, f1_score12))

precision12: 0.5081, recall12: 0.6848, F1-measure: 0.5834


In [44]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction12[k])[0],"\n")

Movie:  End of Days
Actual genre:  mystery, neo noir, murder, stupid, violence, flashback, good versus evil, suspenseful
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Zombeavers
Actual genre:  absurd, entertaining
Predicted tag:  ['violence'] 

Movie:  The Witches of Eastwick
Actual genre:  comedy
Predicted tag:  ['flashback'] 

Movie:  Chain of Desire
Actual genre:  murder
Predicted tag:  [] 

Movie:  Did You Hear About the Morgans?
Actual genre:  entertaining, murder
Predicted tag:  ['flashback' 'murder'] 



<h1>4. TfidfVectorizer with (1 - 4 Grams):</h1>

In [45]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1, 4))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 3)
Dimensions of test data X: (2768, 666) Y: (2768, 3)


<H2> 4.1 OneVsRestClassifier + MultinomialNB :</H2>

In [46]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [47]:
prediction13 = clf.predict(X_test_multilabel)

precision13 = precision_score(y_test_multilabel, prediction13, average='micro')

recall13 = recall_score(y_test_multilabel, prediction13, average='micro')

f1_score13 = 2*((precision13 * recall13)/(precision13 + recall13))

print("precision13: {:.4f}, recall13: {:.4f}, F1-measure: {:.4f}".format(precision13, recall13, f1_score13))

precision13: 0.4680, recall13: 0.6860, F1-measure: 0.5564


In [48]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction13[k])[0],"\n")

Movie:  Hold Your Breath
Actual genre:  violence
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Nae Yeojachinguneun Gumiho
Actual genre:  romantic
Predicted tag:  [] 

Movie:  World Without End
Actual genre:  murder
Predicted tag:  ['violence'] 

Movie:  Absolon
Actual genre:  philosophical
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Dread
Actual genre:  violence, cruelty, murder, flashback
Predicted tag:  ['flashback' 'murder' 'violence'] 



<h2> 4.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</h2>

In [49]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [50]:
prediction14 = clf.predict(X_test_multilabel)

precision14 = precision_score(y_test_multilabel, prediction14, average='micro')

recall14 = recall_score(y_test_multilabel, prediction14, average='micro')

f1_score14 = 2*((precision14 * recall14)/(precision14 + recall14))

print("precision14: {:.4f}, recall14: {:.4f}, F1-measure: {:.4f}".format(precision14, recall14, f1_score14))

precision14: 0.5068, recall14: 0.6864, F1-measure: 0.5831


In [51]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction14[k])[0],"\n")

Movie:  The Spanish Main
Actual genre:  intrigue, action, violence
Predicted tag:  ['violence'] 

Movie:  The Importance of Being Earnest
Actual genre:  romantic
Predicted tag:  [] 

Movie:  Anastasia
Actual genre:  romantic, cute, entertaining
Predicted tag:  ['flashback'] 

Movie:  Flawless
Actual genre:  violence, romantic, murder, storytelling
Predicted tag:  ['flashback' 'murder'] 

Movie:  8MM
Actual genre:  mystery, cruelty, murder, neo noir, cult, revenge, violence, suspenseful, sadist
Predicted tag:  ['murder' 'violence'] 



<h2> 4.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [52]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [53]:
prediction15 = clf.predict(X_test_multilabel)

precision15 = precision_score(y_test_multilabel, prediction15, average='micro')

recall15 = recall_score(y_test_multilabel, prediction15, average='micro')

f1_score15 = 2*((precision15 * recall15)/(precision15 + recall15))

print("precision15: {:.4f}, recall15: {:.4f}, F1-measure: {:.4f}".format(precision15, recall15, f1_score15))

precision15: 0.4954, recall15: 0.6891, F1-measure: 0.5764


In [54]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction15[k])[0],"\n")

Movie:  Elizabeth: The Golden Age
Actual genre:  intrigue
Predicted tag:  ['murder'] 

Movie:  Dishonored 2
Actual genre:  sci-fi
Predicted tag:  ['murder' 'violence'] 

Movie:  In Your Eyes
Actual genre:  paranormal, romantic, fantasy, mystery
Predicted tag:  [] 

Movie:  Final Fantasy: The Spirits Within
Actual genre:  psychedelic, romantic, flashback
Predicted tag:  ['violence'] 

Movie:  From A to Z-Z-Z-Z
Actual genre:  psychedelic
Predicted tag:  [] 



<h2> 4.4 OneVsRestClassifier + LogisticRegression:</h2>

In [55]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [56]:
prediction16 = clf.predict(X_test_multilabel)

precision16 = precision_score(y_test_multilabel, prediction16, average='micro')

recall16 = recall_score(y_test_multilabel, prediction16, average='micro')

f1_score16 = 2*((precision16 * recall16)/(precision16 + recall16))

print("precision16: {:.4f}, recall16: {:.4f}, F1-measure: {:.4f}".format(precision16, recall16, f1_score16))

precision16: 0.5081, recall16: 0.6848, F1-measure: 0.5834


In [57]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction16[k])[0],"\n")

Movie:  Red Sky
Actual genre:  violence, murder
Predicted tag:  [] 

Movie:  Last Summer
Actual genre:  violence, cruelty, romantic
Predicted tag:  [] 

Movie:  Real Life
Actual genre:  satire
Predicted tag:  [] 

Movie:  One Crazy Summer
Actual genre:  psychedelic
Predicted tag:  [] 

Movie:  Mumsy, Nanny, Sonny & Girly
Actual genre:  insanity, cult, murder
Predicted tag:  [] 



<h1> Conclusion: </h1>

In [58]:
from prettytable import PrettyTable

tabel = PrettyTable()

tabel.field_names=['Model','Vectorizer','ngrams','Precision','recall','f1_score']



tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 1)', round(precision1, 3),round(recall1, 3), round(f1_score1, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 1)', round(precision2, 3), round(recall2, 3), round(f1_score2, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 1)' ,round(precision3, 3),  round(recall3, 3), round(f1_score3, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 1)', round(precision4, 3), round(recall4, 3), round(f1_score4, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])


tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 2)', round(precision5, 3), round(recall5, 3),  round(f1_score5, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 2)', round(precision6, 3),  round(recall6, 3),  round(f1_score6, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 2)', round(precision7, 3),  round(recall7, 3),  round(f1_score7, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 2)', round(precision8, 3),  round(recall8, 3),  round(f1_score8, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])


tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 3)', round(precision9, 3),  round(recall9, 3), round(f1_score9, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 3)', round(precision10, 3),  round(recall10, 3), round(f1_score10, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 3)', round(precision11, 3),  round(recall11, 3), round(f1_score11, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 3)', round(precision12, 3),  round(recall12, 3), round(f1_score12, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])

tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 4)', round(precision13, 3), round(recall13, 3), round(f1_score13, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 4)', round(precision14, 3), round(recall14, 3), round(f1_score14, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 4)', round(precision15, 3), round(recall15, 3), round(f1_score15, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 4)', round(precision16, 3),  round(recall16, 3),  round(f1_score16, 3)])



print(tabel)

+----------------------+-----------------+--------+-----------+--------+----------+
|        Model         |    Vectorizer   | ngrams | Precision | recall | f1_score |
+----------------------+-----------------+--------+-----------+--------+----------+
|    MultinomialNB     | TfidfVectorizer | (1, 1) |   0.467   | 0.686  |  0.556   |
|  SGDClassifier(log)  | TfidfVectorizer | (1, 1) |   0.506   | 0.668  |  0.576   |
| SGDClassifier(hinge) | TfidfVectorizer | (1, 1) |   0.511   | 0.661  |  0.576   |
|  LogisticRegression  | TfidfVectorizer | (1, 1) |   0.508   | 0.684  |  0.583   |
|                      |                 |        |           |        |          |
|                      |                 |        |           |        |          |
|    MultinomialNB     | TfidfVectorizer | (1, 2) |   0.468   | 0.686  |  0.556   |
|  SGDClassifier(log)  | TfidfVectorizer | (1, 2) |   0.516   | 0.678  |  0.586   |
| SGDClassifier(hinge) | TfidfVectorizer | (1, 2) |   0.491   | 0.702  |  0.

<h1>5. Word2Vec</h1>

In [59]:
X_train_new = []

for i in tqdm(range(len(list(X_train)))):
    X_train_new.append(X_train[i].split(" "))

100%|█████████████████████████████████████████████████████████████████████████| 10989/10989 [00:00<00:00, 12660.62it/s]


In [60]:
with open('glove.6B.300d.pkl', 'rb') as f:
    new_model = pickle.load(f)
    words =  set(new_model.keys())

In [61]:
X_train_multilabel = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_train.values): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split():
        if word in words:
            vector += new_model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    X_train_multilabel.append(vector)

100%|███████████████████████████████████████████████████████████████████████████| 10989/10989 [00:11<00:00, 939.76it/s]


In [62]:
X_test_multilabel = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_test.values): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split():
        if word in words:
            vector += new_model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    X_test_multilabel.append(vector)

100%|█████████████████████████████████████████████████████████████████████████████| 2768/2768 [00:03<00:00, 877.01it/s]


<H2> 5.1 OneVsRestClassifier + SGDClassifier with LOG Loss :</H2>

In [63]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [64]:
prediction17 = clf.predict(X_test_multilabel)

precision17 = precision_score(y_test_multilabel, prediction17, average='micro')

recall17 = recall_score(y_test_multilabel, prediction17, average='micro')

f1_score17 = 2*((precision17 * recall17)/(precision17 + recall17))

print("precision17: {:.4f}, recall17: {:.4f}, F1-measure: {:.4f}".format(precision17, recall17, f1_score17))

precision17: 0.4235, recall17: 0.7881, F1-measure: 0.5509


In [65]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction17[k])[0],"\n")

Movie:  Topper
Actual genre:  comedy
Predicted tag:  [] 

Movie:  La otra
Actual genre:  murder, melodrama
Predicted tag:  ['flashback' 'murder'] 

Movie:  Conan the Destroyer
Actual genre:  murder, cult, fantasy, alternate history, violence
Predicted tag:  ['violence'] 

Movie:  Magnificent Obsession
Actual genre:  melodrama, flashback
Predicted tag:  ['flashback'] 

Movie:  Lilja 4-ever
Actual genre:  tragedy, violence, depressing
Predicted tag:  ['flashback' 'murder' 'violence'] 



<h2> 5.2 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [66]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [67]:
prediction18 = clf.predict(X_test_multilabel)

precision18 = precision_score(y_test_multilabel, prediction18, average='micro')

recall18 = recall_score(y_test_multilabel, prediction18, average='micro')

f1_score18 = 2*((precision18 * recall18)/(precision18 + recall18))

print("precision18: {:.4f}, recall18: {:.4f}, F1-measure: {:.4f}".format(precision18, recall18, f1_score18))

precision18: 0.4624, recall18: 0.6903, F1-measure: 0.5539


In [68]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction18[k])[0],"\n")

Movie:  Orange County
Actual genre:  flashback
Predicted tag:  [] 

Movie:  Mo' Money
Actual genre:  murder
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Serbuan maut
Actual genre:  cult, suspenseful, murder, violence, flashback
Predicted tag:  ['murder' 'violence'] 

Movie:  Dog Park
Actual genre:  romantic
Predicted tag:  [] 

Movie:  Aan
Actual genre:  romantic
Predicted tag:  [] 



<h2> 5.3 OneVsRestClassifier + LogisticRegression:</h2>

In [69]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [70]:
prediction19 = clf.predict(X_test_multilabel)

precision19 = precision_score(y_test_multilabel, prediction19, average='micro')

recall19 = recall_score(y_test_multilabel, prediction19, average='micro')

f1_score19 = 2*((precision19 * recall19)/(precision19 + recall19))

print("precision19: {:.4f}, recall19: {:.4f}, F1-measure: {:.4f}".format(precision19, recall19, f1_score19))

precision19: 0.4765, recall19: 0.6864, F1-measure: 0.5625


In [71]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction19[k])[0],"\n")

Movie:  In the Line of Fire
Actual genre:  suspenseful, neo noir, murder, mystery, violence, revenge, flashback, good versus evil, humor, action, romantic
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Huang jia shi jie
Actual genre:  violence
Predicted tag:  ['murder'] 

Movie:  The Mirror Crack'd
Actual genre:  melodrama, mystery, satire, murder, flashback
Predicted tag:  ['flashback' 'murder'] 

Movie:  Troll
Actual genre:  fantasy
Predicted tag:  ['violence'] 

Movie:  Doragon bôru Z: Ginga girigiri!! Butchigiri no sugoi yatsu
Actual genre:  violence
Predicted tag:  ['violence'] 



<h1>Conclusion</h1>

In [72]:
from prettytable import PrettyTable

tabel = PrettyTable()

tabel.field_names=['Model', 'Vectorizer', 'Precision','recall','f1_score']


tabel.add_row(['SGDClassifier(log)', 'AVG W2V',  round(precision17, 3), round(recall17, 3), round(f1_score17, 3)])

tabel.add_row(['SGDClassifier(hinge)','AVG W2V', round(precision18, 3),  round(recall18, 3), round(f1_score18, 3)])

tabel.add_row(['LogisticRegression','AVG W2V', round(precision19, 3), round(recall19, 3), round(f1_score19, 3)])


print(tabel)

+----------------------+------------+-----------+--------+----------+
|        Model         | Vectorizer | Precision | recall | f1_score |
+----------------------+------------+-----------+--------+----------+
|  SGDClassifier(log)  |  AVG W2V   |   0.423   | 0.788  |  0.551   |
| SGDClassifier(hinge) |  AVG W2V   |   0.462   |  0.69  |  0.554   |
|  LogisticRegression  |  AVG W2V   |   0.476   | 0.686  |  0.562   |
+----------------------+------------+-----------+--------+----------+


<h1>6. LSTM-CNN Model</h1>

In [73]:
max_review_length = 400
X_train = sequence.pad_sequences(X_train_multilabel, maxlen=max_review_length, padding='post')
X_test = sequence.pad_sequences(X_test_multilabel, maxlen=max_review_length, padding='post')

In [74]:
inputt = 8252
batch_size = 32
epochs = 10

In [75]:
model =Sequential()

model.add(Embedding(inputt, 50, input_length=max_review_length))

model.add(LSTM(100, return_sequences=True))

model.add(Dropout(0.2))

model.add(BatchNormalization())

model.add(LSTM(100, return_sequences=True))

model.add(Dropout(0.2))

model.add(BatchNormalization())

model.add(LSTM(100, return_sequences=True))

model.add(Dropout(0.2))

model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(GlobalMaxPooling1D())

model.add(Dense(3, activation='sigmoid'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 50)           412600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 400, 100)          60400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 100)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 400, 100)          400       
_________________________________________________________________
lstm_2 (LSTM)                (None, 400, 100)          80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 400, 100)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 400, 100)         

In [76]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [77]:
model.fit(X_train, y_train_multilabel, 
          batch_size = batch_size,
          validation_data=(X_test, y_test_multilabel),
          epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 10989 samples, validate on 2768 samples
Epoch 1/10
10989/10989 [==============================] - 664s 60ms/step - loss: 0.9664 - accuracy: 0.2839 - val_loss: 0.9825 - val_accuracy: 0.2840
Epoch 2/10
10989/10989 [==============================] - 658s 60ms/step - loss: 0.9637 - accuracy: 0.2856 - val_loss: 0.9757 - val_accuracy: 0.2840
Epoch 3/10
10989/10989 [==============================] - 658s 60ms/step - loss: 0.9592 - accuracy: 0.2856 - val_loss: 1.0022 - val_accuracy: 0.2840
Epoch 4/10
10989/10989 [==============================] - 656s 60ms/step - loss: 0.9593 - accuracy: 0.2856 - val_loss: 0.9749 - val_accuracy: 0.2840
Epoch 5/10
10989/10989 [==============================] - 654s 60ms/step - loss: 0.9567 - accuracy: 0.2856 - val_loss: 0.9732 - val_accuracy: 0.2840
Epoch 6/10
10989/10989 [==============================] - 653s 59ms/step - loss: 0.9575 - accuracy: 0.2856 - va

In [78]:
test_loss, test_acc = model.evaluate(X_test, y_test_multilabel, verbose=2)

print('\nTest accuracy:', test_acc)


Test accuracy: 0.2839595377445221


In [79]:
model.save('lstm_model_top3.h5') #Saving the Model for Future Use

In [80]:
model = load_model('lstm_model_top3.h5') #Loading the Model

In [81]:
model_prediction = model.predict(X_test, verbose=0)

In [82]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ", y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(model_prediction[k])[0],"\n")

Movie:  Beyond Tomorrow
Actual genre:  romantic, murder
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Rabbit's Kin
Actual genre:  psychedelic
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Zerophilia
Actual genre:  pornographic
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Austin Powers: International Man of Mystery
Actual genre:  comedy, boring, cult, good versus evil, psychedelic, humor, satire, revenge, entertaining
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  The Lost World
Actual genre:  revenge
Predicted tag:  ['flashback' 'murder' 'violence'] 

